In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import json

In [2]:

gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 4m

In [3]:
gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'id'})

In [4]:
gdf_geom : gpd.GeoSeries = gdf.boundary
# 30s

In [5]:
# Combine multiple linestring and multilinestring geometries into single multilinestring geometry
gdml = gdf_geom.apply(lambda x: geom.MultiLineString([x]) if isinstance(x, geom.LineString) else x)
# 1m

In [6]:
gl = gdml.apply(lambda x: x.geoms if x is not None else None).explode()
# 1m

In [7]:
gl.dropna(inplace=True)

In [8]:
coords = gl.apply(lambda x: np.array(x.coords))
# 1m 30s for list, 10s for numpy array

In [9]:
coords.reset_index(drop=True, inplace=True)

In [10]:
coords

0         [[146.92785052248806, -36.083647638487946], [1...
1         [[146.93543157545233, -36.04752350882983], [14...
2         [[146.93040435597695, -36.049962708462175], [1...
3         [[146.9325386953542, -36.0523838486409], [146....
4         [[146.9304386362757, -36.04733293844756], [146...
                                ...                        
371519    [[167.994230239959, -29.045246369874825], [167...
371520    [[150.69854153401263, -35.136031388427355], [1...
371521    [[105.66649220160672, -10.422030053283393], [1...
371522    [[96.88759755192176, -12.110863535868376], [96...
371523    [[167.97281153877768, -29.016557751834803], [1...
Length: 371524, dtype: object

In [39]:
# Get all points
points_full = np.concatenate(coords.to_numpy())
# points_full.sort(axis=0)
len(points_full)
# 19647448
# 10s

19647448

In [12]:
# Drop duplicates
points = np.unique(points_full, axis=0)
len(points)
# 14983534

14983534

In [14]:
# Split long and lat into separate columns, and save to csv, with index and header
df = pd.DataFrame(points, columns=['long', 'lat'])
df.to_csv('local/mb_points.csv', index=True, header=True)
# 1m 30s

In [31]:
df_full = pd.DataFrame(points_full, columns=['long', 'lat'])

In [16]:
pd.read_csv('local/mb_points.csv', index_col=0)
# 10s

,long,lat
0,96.816952,-43.740497
1,96.816952,-43.740469
2,96.816958,-43.740469
3,96.816961,-43.740365
4,96.816970,-43.740365
...,...,...
14983529,167.997444,-9.143163
14983530,167.997528,-9.142913
14983531,167.997742,-9.142483
14983532,167.997797,-9.142293


In [41]:
df_full = pd.DataFrame(points_full, columns=['long', 'lat'])

In [42]:
point0 = coords[0][0]

In [46]:
df_full[(df_full['long'] == point0[0]) & (df_full['lat'] == point0[1])]

,long,lat
0,146.927851,-36.083648
8,146.927851,-36.083648
8343,146.927851,-36.083648
8349,146.927851,-36.083648
14833,146.927851,-36.083648
14844,146.927851,-36.083648


In [47]:
dfx = df_full.drop_duplicates().reset_index(drop=True)
# 20s

In [49]:
dfx[(dfx['long'] == point0[0]) & (dfx['lat'] == point0[1])].index

Index([0], dtype='int64')

In [ ]:
coords.apply(lambda line: [dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in line])
# 0.5s per line

In [55]:
len(coords)

371524

In [52]:
[dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in coords[0]]

[0, 1, 2, 3, 4, 5, 6, 7, 0]

In [54]:
coords[:5].apply(lambda line: [dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in line])

0                          [0, 1, 2, 3, 4, 5, 6, 7, 0]
1    [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...
2    [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 16, 3...
3             [33, 31, 16, 15, 14, 34, 35, 36, 37, 33]
4         [29, 38, 39, 40, 41, 42, 43, 17, 16, 30, 29]
dtype: object